In [1]:
import os
from selenium import webdriver
import pandas as pd
import datetime, time, csv
# import gspread
from bs4 import BeautifulSoup

In [2]:
def get_header_columns(header):
    
    '''
    PortfolioVisualizer Import Benchmark. 
    To generate a list of columns from the webpage HTML file from 
    a table, in order to create a Pandas DataFrame.

    '''
    
    soup = BeautifulSoup(header, 'html.parser')

    columns = []
    initials = []
    index = 0
    for row in soup.findAll('th'):
    #     print(row)
    #     print(row.attrs)
        if 'scope' in row.attrs:
            if 'colspan' in row.attrs:
    #             print('colspan', row['class'], get_initials(row.text.split(' ')))
                initials.append(get_initials(row.text.split(' ')))
            else:
                columns.append(row.text)
        if 'class' in row.attrs and 'group-start' in row['class']:
    #         print (initials[index], row.text)
            columns.append(initials[index]+'_'+row.text)

        if 'class' in row.attrs and 'group-end' in row['class']:
    #         print (initials[index], row.text)
            columns.append(initials[index]+'_'+row.text)
            index += 1
        
    return columns

In [3]:
def get_initials(string_list=[]):
    return "".join(item[0].upper() for item in string_list)

In [ ]:
##### get PV name and password

import getpass

PVNAME = getpass.getpass("Username:")
print (PVNAME)
PVPASSWD = getpass.getpass("Password for " + PVNAME + ":")

Username:········
scubamut@mail.com


# Create Benchmarks

In [4]:
os.environ.update({'PVNAME':'scubamut@gmail.com'})

In [5]:
os.environ.get('PVNAME')

'scubamut@gmail.com'

In [6]:
os.environ.update({'PVPASSWD':'9D@3G!#qH5ZK*k^X*L%x'})

In [50]:
# login to PV

# browser = webdriver.Chrome(executable_path="/usr/lib/chromium-browser/chromedriver",
#                            service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser = webdriver.Chrome(executable_path="/usr/local/share/chromedriver",
                           service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
# browser.set_window_position(-1138,1)
# browser = webdriver.Chrome(executable_path="D:\\chromedriver.exe", service_args=["--verbose", "--log-path=~\Downloads\qc1.log"])
browser.set_window_position(1,1)
# browser.set_window_size(1038, 875)
browser.fullscreen_window()
browser.get('https://www.portfoliovisualizer.com/login')
time.sleep(2)
user = browser.find_element_by_id('username')
user.send_keys(os.environ.get('PVNAME'))
password = browser.find_element_by_id('password')
password.send_keys(os.environ.get('PVPASSWD'))
time.sleep(2)
login = browser.find_element_by_id('submitButton')
login.click()

In [67]:
# add PV assets and year-end, as necessary

assets = {'OOM2000': 'http://bit.ly/2AMTnLO',
          'OOM1987' : 'http://bit.ly/2DmKEgW',
          'ETFMDBS0': 'http://bit.ly/2iN43TU',
          'MDBS00' : 'https://goo.gl/Zzwnhk'} 

# remember that endYear is 2018 so change if necessary

endYear = '2018'

In [68]:
for no,asset in enumerate(assets.keys()):
    print(asset, assets[asset])   
    browser.get(assets[asset])
    browser.find_element_by_id('endYear').clear()
    browser.find_element_by_id('endYear').send_keys(endYear)
    browser.find_element_by_id('submitButton').click()
    browser.find_element_by_xpath('//*[@id="tabs"]/li[4]/a').click()  
    
    monthly_returns = browser.find_element_by_css_selector('#monthlyReturns').text
    results = monthly_returns.splitlines()

    data = [results[3:][i].split(sep=' ') for i in range(len(results[3:]))]
       
    # find table header
    elem = browser.find_element_by_xpath('//*[@id="monthlyReturns"]/div/table/thead')
    header = elem.get_property('outerHTML')
    
    returns = pd.DataFrame(data, columns=get_header_columns(header), index=None)
    returns.Month = [returns.Month.astype(str)[i].zfill(2) for i in range(len(returns))]   
    returns.to_csv('~/Downloads/' + asset + '.csv', index=None)
    
    returns.loc[:,'Period'] = returns.Year.astype(str)+"/"+returns.Month
    
    df = pd.DataFrame( {'Period': returns['Period'], 'Return': returns['R_Timing Portfolio']}, index=None)
    df.to_csv('~/Downloads/' + asset + '.csv', index=None)
    
    data_path = '/home/scubamut/Downloads/'
    symbol = asset
    
    browser.get('https://www.portfoliovisualizer.com/manage-benchmarks#import')
    
    # Series Name
    browser.find_element_by_id("benchmarkName").clear()
    browser.find_element_by_id("benchmarkName").send_keys(asset+'.csv')

    # Upload Data File
    browser.find_element_by_id("upload").clear()
    browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')
    
    # Series (Type)
    browser.find_element_by_id('seriesType_chosen').click()

    # choose SeriesType : li[1|2|3|4] eg li[1] gives Monthly Returns
    browser.find_element_by_xpath('//*[@id="seriesType_chosen"]/div/ul/li[1]').click()    

    # Percentage Values (2=Yes)
    browser.find_element_by_id('percentageValues_chosen').click()
    browser.find_element_by_xpath('//*[@id="percentageValues_chosen"]/div/ul/li[2]').click()

    # Assigned Ticker
    browser.find_element_by_id("benchmarkSymbol").clear()
    browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)
    browser.find_element_by_id('benchmarkAssetClass_chosen').click()
    
    # NOTE: there must be 10 years of monthly data to assign as an Asset Class (2=Yes)
    # But don't need this for creating a benckmark

    # Asset Class (1=Yes)
    browser.find_element_by_css_selector('#benchmarkAssetClass_chosen > div > ul > li:nth-child(1)').click()

    # Import Data Series
    browser.implicitly_wait(90)
    browser.find_element_by_id("importBenchmarkButton").click()
    
    print(browser.find_element_by_class_name("alert").text)

    browser.execute_script("window.history.go(-1)")
    time.sleep(2)
    
browser.close()

OOM2000 http://bit.ly/2AMTnLO
Benchmark series 'OOM2000.csv' imported.
OOM1987 http://bit.ly/2DmKEgW
Benchmark series 'OOM1987.csv' imported.
ETFMDBS0 http://bit.ly/2iN43TU
Benchmark series 'ETFMDBS0.csv' imported.
MDBS00 https://goo.gl/Zzwnhk
Benchmark series 'MDBS00.csv' imported.


# SCRAP

In [52]:
assets

{'OOM2000': 'http://bit.ly/2AMTnLO',
 'OOM1987': 'http://bit.ly/2DmKEgW',
 'ETFMDBS0': 'http://bit.ly/2iN43TU',
 'MDBS00': 'https://goo.gl/Zzwnhk'}

In [53]:
for no,asset in enumerate(assets.keys()):
    print(no,asset, assets[asset])

0 OOM2000 http://bit.ly/2AMTnLO
1 OOM1987 http://bit.ly/2DmKEgW
2 ETFMDBS0 http://bit.ly/2iN43TU
3 MDBS00 https://goo.gl/Zzwnhk


In [54]:
no = 0
asset = 'ETFMDBS0'

In [55]:
browser.get(assets[asset])
browser.find_element_by_id('endYear').clear()
browser.find_element_by_id('endYear').send_keys(endYear)
browser.find_element_by_id('submitButton').click()
browser.find_element_by_xpath('//*[@id="tabs"]/li[4]/a').click() 

In [56]:
monthly_returns = browser.find_element_by_css_selector('#monthlyReturns').text
results = monthly_returns.splitlines()

data = [results[3:][i].split(sep=' ') for i in range(len(results[3:]))]

# find table header
elem = browser.find_element_by_xpath('//*[@id="monthlyReturns"]/div/table/thead')
header = elem.get_property('outerHTML')

In [57]:
returns = pd.DataFrame(data, columns=get_header_columns(header), index=None)
returns.Month = [returns.Month.astype(str)[i].zfill(2) for i in range(len(returns))]   
returns.to_csv('~/Downloads/' + asset + '.csv', index=None)

returns.loc[:,'Period'] = returns.Year.astype(str)+"/"+returns.Month

df = pd.DataFrame( {'Period': returns['Period'], 'Return': returns['R_Timing Portfolio']}, index=None)
df.to_csv('~/Downloads/' + asset + '.csv', index=None)

In [58]:
data_path = '/home/scubamut/Downloads/'
symbol = asset

In [59]:
browser.get('https://www.portfoliovisualizer.com/manage-benchmarks#import')

In [60]:
# Series Name
browser.find_element_by_id("benchmarkName").clear()
browser.find_element_by_id("benchmarkName").send_keys(asset+'.csv')

In [61]:
# Upload Data File
browser.find_element_by_id("upload").clear()
browser.find_element_by_id("upload").send_keys(data_path + symbol +'.csv')


In [62]:
# Series (Type)
browser.find_element_by_id('seriesType_chosen').click()

# choose SeriesType : li[1|2|3|4] eg li[1] gives Monthly Returns
browser.find_element_by_xpath('//*[@id="seriesType_chosen"]/div/ul/li[1]').click()

In [63]:
# 1=No, 2=Yes
browser.find_element_by_id('percentageValues_chosen').click()
browser.find_element_by_xpath('//*[@id="percentageValues_chosen"]/div/ul/li[2]').click()

In [64]:
browser.find_element_by_id("benchmarkSymbol").clear()
browser.find_element_by_id("benchmarkSymbol").send_keys(symbol)
browser.find_element_by_id('benchmarkAssetClass_chosen').click()

In [65]:
browser.find_element_by_css_selector('#benchmarkAssetClass_chosen > div > ul > li:nth-child(2)').click()

In [66]:
browser.implicitly_wait(90)
browser.find_element_by_id("importBenchmarkButton").click()